In [ ]:
## OpenSoundscape RIBBIT tutorial
# http://opensoundscape.org/en/latest/tutorials/RIBBIT_pulse_rate_demo.html

## Setup 

In [3]:
# suppress warnings
import warnings
warnings.simplefilter('ignore')
warnings.simplefilter('ignore', UserWarning)

#import packages
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import subprocess
from datetime import datetime


#local imports from opensoundscape
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from opensoundscape.ribbit import ribbit

# create big visuals
plt.rcParams['figure.figsize']=[15,8]
pd.set_option('display.precision', 2)

## Set parameters 

In [ ]:
# These parameters can be adjusted to try to improve the model or to change the model for a different herp species 

In [4]:
# minimum and maximum rate of pulsing (pulses per second) to search for
pulse_rate_range = [14,28] 

# look for a vocalization in the range of 600-800 Hz
signal_band = [600,800]

# subtract the amplitude signal from these frequency ranges
noise_bands = [[150,200], [1500,1600]]

#divides the signal into segments this many seconds long, analyzes each independently
clip_duration = 1 #seconds
clip_overlap = 0.5 #seconds

#if True, it will show the power spectrum plot for each audio segment
show_plots = False 


In [5]:
# run RIBBIT for a specific every index in a df where the index is a file path; save the score in that df 
def run_ribbit_for_indices(df):
    start_time = datetime.now() # track how long the model takes
    checkpoint = 0; # create checkpoint for saving RIBBIT scores periodically 

    # for every data file for year i, pond j... calculate RIBBIT score 
    for path in df.index:

        try: # make sure RIBBIT is working for this audio file, if it doesn't work for some reason, skip this so the model will continue running 
            #make the spectrogram
            spec = Spectrogram.from_audio(Audio.from_file(path))

            #run RIBBIT on the spectrogram 
            score_df =  ribbit(
                                spec,
                                pulse_rate_range=pulse_rate_range,
                                signal_band=signal_band,
                                clip_duration=clip_duration,
                                noise_bands=noise_bands,
                                plot=False)


            maxIndex= score_df.score.idxmax() # find the index for where the max RIBBIT score occurs for the audio file 

            # record the max RIBBIT score for the auio file (multiplied by 10,000 to make it easier to read)
            df.at[path,'score'] = score_df.at[maxIndex, 'score'] * 10000
            # record the time in the audio file that the max RIBBIT score occurs (so can be checked manually)
            df.at[path,'time_stamp'] = score_df.at[maxIndex, 'start_time']

            checkpoint = checkpoint + 1 

        except (RuntimeError, EOFError) as runtime_error: # if the file does not work with RIBBIT, skip it and output a failure message; use -1 for RIBBIT score and time stamp
            print("File failed: ", path)
            print(runtime_error)


            df.at[path,'score'] = -1 
            df.at[path,'time_stamp'] = -1
            checkpoint = checkpoint + 1
            pass 

        # every 100 files, print checkpoint message and save RIBBIT scores to csv file to back them up 
        if checkpoint % 100 == 0:
            print("Finished file number ", checkpoint, "/nFile name: ", path)
            print(df.sort_values(by='score',ascending=False).head(1))
            df.to_csv(csv_file_name) 


    df.sort_values(by='score',ascending=False).to_csv(csv_file_name) # sort by highest RIBBIT scores and save to csv
    print('Time elapsed ',datetime.now() - start_time) # print time taken for this pond-year

    #print("Files sorted by score, from highest to lowest:")
    df.sort_values(by='score',ascending=False)   

        

## Analyzing all files

In [6]:
warnings.filterwarnings("ignore")

#took about 35  min for one pond, one year
all_years = [str(x) for x in range(2015, 2016)] # choose years to analyze 
all_ponds = ["1a"]#["1a", "2a", "3a", "4a", "5a", "6a", "7a", "8a", "9a", "10a"] # choose pond number to analyze 
all_data = list() # setup variable to save RIBBIT scores 

# set up a dataframe for storing files' scores and labels
for i in range(0, len(all_years)):
    pond_data = list() 
    for j in range(0, len(all_ponds)):
        
        # folder name where data is stored
        folder = '/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_' + all_years[i] + '/' + all_ponds[j] + '/*'
        
        # name for csv file to save RIBBIT scores  
        csv_file_name = './ribbit_scores/' + 'ichaway_'+ all_ponds[j] + '_' + all_years[i] + '_ribbit_scores.csv'

        
        df = pd.DataFrame(index = glob(folder),columns=['score', 'year', 'pond']) # create dataframe for year i, pond j
        df['year'] = all_years[i] # automatically label year for these files
        df['pond'] = all_ponds[j] # automatically label pond for these files

        start_time = datetime.now() # track how long the model takes
        checkpoint = 0; # create checkpoint for saving RIBBIT scores periodically 
        
        # for every data file for year i, pond j... calculate RIBBIT score 
        run_ribbit_for_indices(df)
        
        #print("Files sorted by score, from highest to lowest:")
        df.sort_values(by='score',ascending=False)
        pond_data.append(df) # add year i to the pond_data
        
    all_data.append(pond_data) # add pond j to all_data 
    
    
# all_data is a list object - this translates it into a single datafraome
temp_list = []
for i in range(0,len(all_data)):
    temp_df = pd.concat(all_data[i], axis=0, ignore_index=False) # translates list into dataframe 
    temp_list.append(temp_df)
    
all_data_df = pd.concat(temp_list, axis=0, ignore_index=False) # translates list into dataframe 
all_data_df
all_data_df.to_csv('./ribbit_scores/ichaway_all_files_ribbit_scores.csv') # save all_data_df to csv

    


/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 63, using nperseg = 63
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 63, using nperseg = 63
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

KeyboardInterrupt: 